# Initialization

In [1]:
# data processing
import numpy as np
import pandas as pd 
from collections import defaultdict

# data visualization
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
sns.set()
import urllib.request


import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
from tensorflow import keras

# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# If using Google Colab
from google.colab import drive
BASE_PATH = '/content/drive'
drive.mount(BASE_PATH)

# change directory
import os
PROJECT_PATH = os.path.join(BASE_PATH, "MyDrive", "ECBM4040", "FinalProject", "recreating-residual-attention-network")
os.chdir(PROJECT_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#adjust filepath so that working directory is folder "recreating-residual-attention-network"
%pwd
%cd .. 

C:\Users\free_\OneDrive\Documents\GitHub\recreating-residual-attention-network


In [9]:
# Import created modules
from src.models.ResidualAttentionNetwork import ResidualAttentionNetwork, Attention56, Attention92, Attention128, Attention164
from src.utils import generate_data

# Modelling

In [10]:
INPUT_SHAPE, NUM_CLASS, train_ds, val_ds, test_ds, _ = generate_data.get_cifar10()

In [5]:
BATCH_SIZE = 256
N_EPOCH = 10


In [11]:
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.shuffle(buffer_size=BATCH_SIZE*4).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [7]:
!nvidia-smi

Sat Dec  4 21:34:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    25W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Naive Attention Learning vs Residual Attention Learning

### Naive Attention Learning

In [8]:
ran_model = ResidualAttentionNetwork(input_shape=INPUT_SHAPE, num_class=NUM_CLASS, learning_type='nal')
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "residual_attention_network"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             multiple                  864       
                                                                 
 batch_normalization (BatchN  multiple                 128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                multiple                  0         
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 residual_unit (ResidualUnit  multiple  

In [9]:
nal_model = ran_model

nal_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = nal_model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 37s 110ms/step - loss: 2.4133 - accuracy: 0.1461 - val_loss: 2.5617 - val_accuracy: 0.1000
Epoch 2/10
157/157 [==============================] - 15s 95ms/step - loss: 2.2672 - accuracy: 0.1830 - val_loss: 3.2553 - val_accuracy: 0.0950
Epoch 3/10
157/157 [==============================] - 15s 95ms/step - loss: 2.1580 - accuracy: 0.2144 - val_loss: 2.3432 - val_accuracy: 0.2005
Epoch 4/10
157/157 [==============================] - 15s 95ms/step - loss: 2.0650 - accuracy: 0.2461 - val_loss: 2.1246 - val_accuracy: 0.2459
Epoch 5/10
157/157 [==============================] - 15s 96ms/step - loss: 1.9930 - accuracy: 0.2685 - val_loss: 2.2023 - val_accuracy: 0.2280
Epoch 6/10
157/157 [==============================] - 15s 95ms/step - loss: 1.9244 - accuracy: 0.2909 - val_loss: 2.0788 - val_accuracy: 0.2697
Epoch 7/10
157/157 [==============================] - 15s 95ms/step - loss: 1.8702 - accuracy: 0.3154 - val_loss: 1.8306 - val_accuracy

### Residual Attention Learning

In [10]:
ran_model = ResidualAttentionNetwork(input_shape=INPUT_SHAPE, num_class=NUM_CLASS, learning_type='arl')
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "residual_attention_network_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32, 32, 3)         0         
                                                                 
 conv2d_139 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_107 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_107 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_7 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 residual_unit_33 (ResidualU  multiple

In [ ]:
arl_model = ran_model

arl_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = arl_model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 64s 306ms/step - loss: 2.4113 - accuracy: 0.1523 - val_loss: 2.5067 - val_accuracy: 0.1000
Epoch 2/10
157/157 [==============================] - 45s 288ms/step - loss: 2.1210 - accuracy: 0.2495 - val_loss: 2.5251 - val_accuracy: 0.1268
Epoch 3/10
157/157 [==============================] - 45s 288ms/step - loss: 1.9903 - accuracy: 0.2894 - val_loss: 2.1483 - val_accuracy: 0.2473
Epoch 4/10
157/157 [==============================] - 46s 290ms/step - loss: 1.8977 - accuracy: 0.3189 - val_loss: 1.9694 - val_accuracy: 0.2904
Epoch 5/10
157/157 [==============================] - 45s 289ms/step - loss: 1.8269 - accuracy: 0.3391 - val_loss: 1.7982 - val_accuracy: 0.3653
Epoch 6/10
157/157 [==============================] - 45s 288ms/step - loss: 1.7730 - accuracy: 0.3625 - val_loss: 1.7727 - val_accuracy: 0.3656
Epoch 7/10
157/157 [==============================] - 45s 289ms/step - loss: 1.7417 - accuracy: 0.3726 - val_loss: 1.7011 - val_ac

## Different number of attention module

In this part, the experiment is done using Residual Attention Learning mechanism

### Attention-56

In [16]:
ran_model = Attention56()
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "attention56_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_139 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_107 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_107 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_7 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 residual_unit_33 (ResidualU  multiple               

In [17]:
a56_model = ran_model

a56_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = a56_model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=N_EPOCH, verbose=1)

Epoch 1/10


KeyboardInterrupt: ignored

### Attention-92

In [13]:
ran_model = Attention92()
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "attention92"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_417 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_321 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_321 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_21 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 residual_unit_99 (ResidualU  multiple                 

In [14]:
a92_model = ran_model

a92_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss=tf.keras.losses.CategoricalCrossentropy(), 
                  metrics=['accuracy'])


history = a92_model.fit(train_ds,
                        validation_data=val_ds,
                        epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 68s 297ms/step - loss: 2.4021 - accuracy: 0.1148 - val_loss: 2.3035 - val_accuracy: 0.1000
Epoch 2/10
157/157 [==============================] - 43s 276ms/step - loss: 2.3327 - accuracy: 0.1309 - val_loss: 2.3277 - val_accuracy: 0.1000
Epoch 3/10
157/157 [==============================] - 43s 277ms/step - loss: 2.3142 - accuracy: 0.1392 - val_loss: 2.3169 - val_accuracy: 0.1000
Epoch 4/10
157/157 [==============================] - 43s 276ms/step - loss: 2.3007 - accuracy: 0.1388 - val_loss: 2.2687 - val_accuracy: 0.1425
Epoch 5/10
157/157 [==============================] - 43s 276ms/step - loss: 2.2841 - accuracy: 0.1444 - val_loss: 2.2515 - val_accuracy: 0.1477
Epoch 6/10
157/157 [==============================] - 43s 276ms/step - loss: 2.2747 - accuracy: 0.1508 - val_loss: 2.3620 - val_accuracy: 0.1074
Epoch 7/10
157/157 [==============================] - 43s 276ms/step - loss: 2.2591 - accuracy: 0.1549 - val_loss: 2.6244 - val_ac

### Attention-128

In [15]:
ran_model = Attention128()
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "attention128"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_670 (Conv2D)         multiple                  864       
                                                                 
 batch_normalization_515 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_515 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_34 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 residual_unit_159 (Residual  multiple                

In [16]:
a128_model = ran_model

a128_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                   loss=tf.keras.losses.CategoricalCrossentropy(), 
                   metrics=['accuracy'])


history = a128_model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 97s 425ms/step - loss: 2.3997 - accuracy: 0.1120 - val_loss: 2.3032 - val_accuracy: 0.1000
Epoch 2/10
157/157 [==============================] - 62s 394ms/step - loss: 2.3201 - accuracy: 0.1385 - val_loss: 2.3081 - val_accuracy: 0.1000
Epoch 3/10
157/157 [==============================] - 62s 394ms/step - loss: 2.3025 - accuracy: 0.1425 - val_loss: 2.3207 - val_accuracy: 0.1014
Epoch 4/10
157/157 [==============================] - 62s 393ms/step - loss: 2.2850 - accuracy: 0.1495 - val_loss: 2.2386 - val_accuracy: 0.1558
Epoch 5/10
157/157 [==============================] - 62s 393ms/step - loss: 2.2778 - accuracy: 0.1456 - val_loss: 2.2371 - val_accuracy: 0.1636
Epoch 6/10
157/157 [==============================] - 62s 394ms/step - loss: 2.2649 - accuracy: 0.1521 - val_loss: 2.3654 - val_accuracy: 0.1311
Epoch 7/10
157/157 [==============================] - 62s 393ms/step - loss: 2.2690 - accuracy: 0.1529 - val_loss: 2.3923 - val_ac

### Attention-164

In [ ]:
ran_model = Attention164()
inputs = tf.keras.Input((32, 32, 3))
ran_model(inputs)
ran_model.summary()

Model: "attention164"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (None, 32, 32, 3)         0         
                                                                 
 conv2d_1037 (Conv2D)        multiple                  864       
                                                                 
 batch_normalization_796 (Ba  multiple                 128       
 tchNormalization)                                               
                                                                 
 re_lu_796 (ReLU)            multiple                  0         
                                                                 
 max_pooling2d_53 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 residual_unit_246 (Residual  multiple                

In [17]:
a164_model = ran_model

a164_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                   loss=tf.keras.losses.CategoricalCrossentropy(), 
                   metrics=['accuracy'])


history = a164_model.fit(train_ds,
                         validation_data=val_ds,
                         epochs=N_EPOCH, verbose=1)

Epoch 1/10
157/157 [==============================] - 98s 425ms/step - loss: 2.1079 - accuracy: 0.1941 - val_loss: 2.1368 - val_accuracy: 0.1748
Epoch 2/10
157/157 [==============================] - 62s 395ms/step - loss: 1.9617 - accuracy: 0.2410 - val_loss: 2.9659 - val_accuracy: 0.1587
Epoch 3/10
157/157 [==============================] - 62s 395ms/step - loss: 1.9016 - accuracy: 0.2630 - val_loss: 3.4470 - val_accuracy: 0.1594


KeyboardInterrupt: ignored

## Wide ResNet Comparison